In [1]:
import os
import re
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from scipy import stats
import func4behav as fb
from scipy.stats import lognorm, exponnorm, invgauss
from scipy.optimize import curve_fit
from sklearn.metrics import mean_squared_error
from pyddm import Model, Sample, Fittable, Fitted
from pyddm.models import DriftConstant, NoiseConstant, BoundConstant, OverlayNonDecision, ICPointSourceCenter, LossRobustLikelihood
from pyddm.functions import fit_adjust_model, display_model
import pyddm.plot

import imp
imp.reload(fb)

C:\Users\Tianyi Zheng\AppData\Local\Temp\ipykernel_27136\484827349.py:17: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  import imp


<module 'func4behav' from 'c:\\Users\\Tianyi Zheng\\OneDrive - neuron.t.u-tokyo.ac.jp\\Documents\\zheng\\mywork\\attention_tES\\tes-attention\\processing\\behavior\\func4behav.py'>

In [2]:
def distribution_param(case_list, cutoff):

    _, expt_info = fb.create_allsubs_compare()
    param_record = pd.DataFrame(columns=['subject id', 'case', 'group', 'session', 'method', 'params', 'log likelihood', 'AIC'])
    
    session_name = ['before', 'after']
    methods_name = ['shift lognorm', 'exgaussian', 'ddm', 'shift invgauss']
    methods = [fb.fit_shift_lognorm, fb.fit_exgaussian, fb.fit_ddm, fb.fit_shift_invgauss]

    # Loops
    record_idx = 0
    for subject_id in range(1,19):
        print("Subject id:" + str(subject_id) + " is processing...")
        real_or_sham = expt_info.loc[expt_info['subject id'] == subject_id, 'Real stimulation'].values[0]

        for case in case_list:
            # print("Case:" + str(case) + " is processing...")
            rt_before, rt_after = make_rt_data(subject_id, case, cutoff)

            for method_idx, method in enumerate(methods):
                for session_idx, rt in enumerate([rt_before, rt_after]):
                    param, log_L, AIC = method(rt, cutoff)

                    param_record.loc[record_idx, 'subject id'] = subject_id
                    param_record.loc[record_idx, 'case'] = case
                    param_record.loc[record_idx, 'group'] = "real" if real_or_sham == 1 else "sham"
                    param_record.loc[record_idx, 'session'] = session_name[session_idx]
                    param_record.loc[record_idx, 'method'] = methods_name[method_idx]
                    param_record.loc[record_idx, 'params'] = param
                    param_record.loc[record_idx, 'log likelihood'] = log_L
                    param_record.loc[record_idx, 'AIC'] = AIC

                    record_idx += 1
                    
    return param_record


def make_rt_data(subject_id, case, cutoff):
    
    behavior_before, behavior_after = fb.load_behavior(subject_id)
    behavior_before, behavior_after = fb.filter_behav(case, behavior_before, behavior_after)
    rt_before = behavior_before.loc[(behavior_before['response'] == 1) & (behavior_before['reaction time'] > 0.05) & (behavior_before['reaction time'] < cutoff)]
    rt_after = behavior_after.loc[(behavior_after['response'] == 1) & (behavior_after['reaction time'] > 0.05) & (behavior_after['reaction time'] < cutoff)]

    rt_before = rt_before['reaction time'].values
    rt_after = rt_after['reaction time'].values

    return rt_before, rt_after

In [3]:
all_real = [1, 3, 4, 5, 9, 12, 13, 17, 18]
all_sham = [2, 6, 7, 8, 10, 11, 14, 15, 16]
case_list = ['all', 'endo', 'exo', 'valid', 'endo valid', 'exo valid', 'invalid', 'endo invalid', 'exo invalid', 'stim left', 'endo stim left', 'endo stim right', 'stim right', 'exo stim left', 'exo stim right']
cutoff = 1
# param_record = distribution_param(case_list, cutoff)
# param_record.to_csv('../../../docs/report/data/param_record.csv', index=False)

## compute P value

In [4]:
param_record_path = os.path.join('..', '..', '..', 'docs', 'report', 'data', 'param_record.csv')
param_record = pd.read_csv(param_record_path)
param_record

,subject id,case,group,session,method,params,log likelihood,AIC
0,1,all,real,before,shift lognorm,"[0.530450629232766, 0.09696554056594511, 0.119...",184.279441,-362.558881
1,1,all,real,after,shift lognorm,"[0.1727647541500616, 0.3451221542342069, -0.15...",162.489464,-318.978928
2,1,all,real,before,exgaussian,"[2.9564068064090367, 0.1736784921094102, 0.019...",185.574583,-365.149166
3,1,all,real,after,exgaussian,"[1.1293728835241597, 0.1461682210666338, 0.040...",164.208207,-322.416415
4,1,all,real,before,ddm,"[9.69515774149879, 1.7652210978910852, 1.16170...",182.518774,-357.037547
...,...,...,...,...,...,...,...,...
2155,18,exo stim right,real,after,exgaussian,"[4.223375125196021, 0.24303794121277486, 0.014...",48.557894,-91.115789
2156,18,exo stim right,real,before,ddm,"[8.798864077268108, 1.4167599334854657, 2.0381...",36.628143,-65.256286
2157,18,exo stim right,real,after,ddm,"[9.961070584448608, 1.8745560474589342, 0.9924...",48.672956,-89.345912
2158,18,exo stim right,real,before,shift invgauss,"[0.11336152603077676, 0.10825551392430313, 2.0...",36.628786,-67.257573


In [5]:
def compute_p(data1, data2, p_method):
    if p_method == 1:
        _, p_value = stats.mannwhitneyu(data1, data2)
    else:
        _, p_value = stats.ttest_ind(data1, data2)
    return p_value


def get_p_values(df, param_number, p_method):
    p_values = [None] * param_number
    param_before = df.loc[df['session'] == 'before', 'params']
    param_after = df.loc[df['session'] == 'after', 'params']
    for i in range(param_number):
        before = [float(x.split(',')[i].strip('[]')) for x in param_before.astype(str)]
        after = [float(x.split(',')[i].strip('[]')) for x in param_after.astype(str)]
        p_values[i] = round(compute_p(before, after, p_method), 3)

    return '[' + ', '.join([str(x) for x in p_values]) + ']'

In [6]:
methods_name = ['shift lognorm', 'exgaussian', 'ddm', 'shift invgauss']
method_params = [3,3,4,3]
p_table = pd.DataFrame(columns=['case', methods_name[0] + ': sham', methods_name[0] + ': real', methods_name[1] + ': sham', methods_name[1] + ': real',
                                 methods_name[2] + ': sham', methods_name[2] + ': real', methods_name[3] + ': sham', methods_name[3] + ': real'])
p_table['case'] = case_list

In [7]:
for case in case_list:
    df_case = param_record.loc[param_record['case'] == case]
    for group in ['sham', 'real']:
        df_group = df_case.loc[df_case['group'] == group]
        for i, method in enumerate(methods_name):
            df_method = df_group.loc[df_group['method'] == method]
            p_values = get_p_values(df_method, method_params[i], p_method=2)
            p_table.loc[p_table['case'] == case, method+': '+group] = p_values

In [8]:
p_table

,case,shift lognorm: sham,shift lognorm: real,exgaussian: sham,exgaussian: real,ddm: sham,ddm: real,shift invgauss: sham,shift invgauss: real
0,all,"[0.842, 0.52, 0.401]","[0.832, 0.908, 0.879]","[0.787, 0.529, 0.612]","[0.923, 0.561, 0.78]","[0.85, 0.746, 0.731, 0.765]","[0.807, 0.337, 0.679, 0.978]","[0.826, 0.517, 0.334]","[0.742, 0.895, 0.978]"
1,endo,"[0.944, 0.491, 0.447]","[0.775, 0.787, 0.598]","[0.748, 0.483, 0.537]","[0.652, 0.67, 0.77]","[0.113, 0.286, 0.042, 0.613]","[0.657, 0.676, 0.868, 0.661]","[0.997, 0.881, 0.66]","[0.72, 0.583, 0.555]"
2,exo,"[0.322, 0.955, 0.948]","[0.989, 0.343, 0.346]","[0.309, 0.593, 0.536]","[0.335, 0.396, 0.588]","[0.602, 0.174, 0.631, 0.154]","[0.331, 0.995, 0.078, 0.3]","[0.359, 0.359, 0.465]","[0.702, 0.657, 0.414]"
3,valid,"[0.87, 0.728, 0.59]","[0.815, 0.955, 0.741]","[0.823, 0.553, 0.486]","[0.899, 0.566, 0.863]","[0.23, 0.23, 0.352, 0.705]","[0.588, 0.893, 0.568, 0.702]","[0.783, 0.831, 0.735]","[0.879, 0.772, 0.845]"
4,endo valid,"[0.865, 0.366, 0.356]","[0.895, 0.351, 0.302]","[0.902, 0.492, 0.496]","[0.194, 0.604, 0.507]","[0.853, 0.517, 0.332, 0.82]","[0.361, 0.615, 0.846, 0.508]","[0.933, 0.831, 0.661]","[0.788, 0.383, 0.282]"
5,exo valid,"[0.657, 0.375, 0.388]","[0.968, 0.466, 0.473]","[0.342, 0.501, 0.817]","[0.34, 0.365, 0.382]","[0.126, 0.93, 0.197, 0.268]","[0.812, 0.703, 0.085, 0.238]","[0.981, 0.51, 0.875]","[0.691, 0.678, 0.438]"
6,invalid,"[0.223, 0.496, 0.492]","[0.927, 0.598, 0.599]","[0.151, 0.735, 0.338]","[0.152, 0.839, 0.873]","[0.14, 0.125, 0.424, 0.525]","[0.594, 0.743, 0.29, 0.635]","[0.121, 0.411, 0.506]","[0.776, 0.895, 0.884]"
7,endo invalid,"[0.882, 0.724, 0.712]","[0.313, 0.501, 0.501]","[0.663, 0.579, 0.645]","[0.07, 0.935, 0.586]","[0.729, 0.516, 0.447, 0.887]","[0.385, 0.764, 0.707, 0.658]","[0.885, 0.82, 0.858]","[0.191, 0.651, 0.509]"
8,exo invalid,"[0.103, 0.027, 0.026]","[0.057, 0.739, 0.744]","[0.623, 0.606, 0.45]","[0.04, 0.433, 0.063]","[0.338, 0.901, 0.026, 0.042]","[0.76, 0.068, 0.166, 0.262]","[0.192, 0.026, 0.025]","[0.037, 0.292, 0.2]"
9,stim left,"[0.441, 0.357, 0.933]","[0.795, 0.801, 0.875]","[0.607, 0.43, 0.192]","[0.333, 0.585, 0.489]","[0.351, 0.889, 0.51, 0.95]","[0.394, 0.414, 0.565, 0.751]","[0.22, 0.936, 0.552]","[0.64, 0.707, 0.479]"
